In [15]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
%matplotlib qt

try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
except:
    from tkinter import Tk
    from tkinter import filedialog

In [16]:
def gaussian(x, N, mu, sigma):
    return N*np.exp(-0.5*((x-mu)/sigma)**2)

def doubleGaussian(x, N1, mu1, sigma1, N2, mu2, sigma2):
    return N1*np.exp(-0.5*((x-mu1)/sigma1)**2) + N2*np.exp(-0.5*((x-mu2)/sigma2)**2) 


#### Calibration

In [17]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
calib_file_name = filedialog.askopenfilenames(parent = root) 

In [18]:

chanel_cal, energy_cal = np.loadtxt(calib_file_name[0], unpack = 'True')

#NB: Ordered from n-th order to 0-th order
calib_coeff = np.polyfit(chanel_cal, energy_cal, deg = 1)

In [24]:
mock_channel = np.linspace(0,2000,2000)
cal_line = mock_channel*calib_coeff[0] + calib_coeff[1]

fig = plt.figure()
ax = fig.add_subplot()

ax.set_title('Calibration Plot')
ax.plot(mock_channel, cal_line, color = 'grey', label = 'Cal. Line')
ax.scatter(chanel_cal, energy_cal, marker = '+', color ='red', label = 'Cal. Points')
ax.set_xlabel('Channel')
ax.set_ylabel('Energy [keV]')
ax.legend()
ax.grid()


#### Data Analysis

In [25]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 
data_file_name = filedialog.askopenfilenames(parent = root) 

In [27]:
chanel, counts = np.loadtxt(data_file_name[0], unpack = True, skiprows=10, delimiter =';', usecols=(0,1))
calib_energies =  chanel*calib_coeff[0] + calib_coeff[1]


In [41]:
fig, ax = plt.subplots(1,2, figsize = (15,7))
#ax = fig.add_subplot(nrows = 1,ncols = 2)

ax[0].plot(chanel, counts, color = 'gray')
ax[0].set_xlabel('Channel')
ax[0].set_ylabel('Counts')
ax[0].grid()

ax[1].plot(calib_energies, counts, color = 'gray')
ax[1].set_xlabel('Energy [keV]')
ax[1].set_ylabel('Counts')
ax[1].grid()